# Street Fighter No Delta
This notebook shows how to create the environment without the delta transformation. 

# Setup StreetFighter

In [2]:
# !pip install gym gym-retro

In [3]:
import retro
import retrowrapper

In [4]:
retro.data.list_games()

['1942-Nes',
 '1943-Nes',
 '3NinjasKickBack-Genesis',
 '8Eyes-Nes',
 'AaahhRealMonsters-Genesis',
 'AbadoxTheDeadlyInnerWar-Nes',
 'AcceleBrid-Snes',
 'ActRaiser2-Snes',
 'ActionPachio-Snes',
 'AddamsFamily-GameBoy',
 'AddamsFamily-Genesis',
 'AddamsFamily-Nes',
 'AddamsFamily-Sms',
 'AddamsFamily-Snes',
 'AddamsFamilyPugsleysScavengerHunt-Nes',
 'AddamsFamilyPugsleysScavengerHunt-Snes',
 'AdvancedBusterhawkGleylancer-Genesis',
 'Adventure-Atari2600',
 'AdventureIsland-GameBoy',
 'AdventureIsland3-Nes',
 'AdventureIslandII-Nes',
 'AdventuresOfBatmanAndRobin-Genesis',
 'AdventuresOfBayouBilly-Nes',
 'AdventuresOfDinoRiki-Nes',
 'AdventuresOfDrFranken-Snes',
 'AdventuresOfKidKleets-Snes',
 'AdventuresOfMightyMax-Genesis',
 'AdventuresOfMightyMax-Snes',
 'AdventuresOfRockyAndBullwinkleAndFriends-Genesis',
 'AdventuresOfRockyAndBullwinkleAndFriends-Nes',
 'AdventuresOfRockyAndBullwinkleAndFriends-Snes',
 'AdventuresOfStarSaver-GameBoy',
 'AdventuresOfYogiBear-Snes',
 'AeroFighters-Snes',
 

In [5]:
!python -m retro.import .

Importing StreetFighterIISpecialChampionEdition-Genesis
Importing Airstriker-Genesis
Importing StreetFighterIISpecialChampionEdition-Genesis
Imported 3 games


In [6]:
gamename = 'StreetFighterIISpecialChampionEdition-Genesis'
# env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')
env = retrowrapper.RetroWrapper(gamename, use_restricted_actions=retro.Actions.FILTERED)

In [7]:
obs = env.reset()

In [24]:
obs = env.reset()
done = False
for game in range(5):
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        print(reward)

KeyboardInterrupt: 

In [9]:
# https://wowroms.com/en/roms/sega-genesis-megadrive/download-street-fighter-ii-special-champion-edition-europe/26496.html

In [10]:
env.close()

In [23]:
env.action_space.sample()

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0], dtype=int8)

In [11]:
env.observation_space.sample()

array([[[176, 167, 221],
        [141,  36, 171],
        [ 35, 251, 223],
        ...,
        [ 42,  32,  38],
        [242, 181, 166],
        [ 71, 162,  80]],

       [[  9, 155, 124],
        [130,  34,  40],
        [143, 184, 130],
        ...,
        [ 51, 207, 227],
        [125,  59, 156],
        [198,  55,  25]],

       [[217,  71, 130],
        [ 30,  46,   9],
        [  9,  64,  63],
        ...,
        [177, 223,  87],
        [241, 193,  13],
        [135, 124, 210]],

       ...,

       [[ 49, 152, 139],
        [ 23,  61, 235],
        [177, 143, 102],
        ...,
        [151,  23, 166],
        [251, 176, 150],
        [ 55,  87, 174]],

       [[222, 177, 129],
        [104, 112, 176],
        [111,  45,  69],
        ...,
        [105,  23,  88],
        [208,  97,  40],
        [149, 203, 161]],

       [[223, 189, 177],
        [ 25,  72,  53],
        [141, 160,  21],
        ...,
        [ 90,   3, 162],
        [246,  80, 147],
        [193, 105, 245]]

In [12]:
# info['enemy_health']

In [13]:
# info['health']

# Setup Environment

In [14]:
from gym import Env
from gym.spaces import Box, MultiBinary
import numpy as np
import cv2

In [15]:
class StreetFighter(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        # self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
        self.game = retrowrapper.RetroWrapper(gamename, use_restricted_actions=retro.Actions.FILTERED)
        # self.score = 0
    
    def step(self, action):
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)
        
        # Preprocess frame from game
        frame_delta = obs 
#         - self.previous_frame
#         self.previous_frame = obs 
        
        # Shape reward
        reward = info['score'] - self.score 
        self.score = info['score']

        return frame_delta, reward, done, info 
    
    def render(self, *args, **kwargs): 
        self.game.render()
    
    def reset(self):
        self.previous_frame = np.zeros(self.game.observation_space.shape)
        
        # Frame delta
        obs = self.game.reset()
        obs = self.preprocess(obs)
        self.previous_frame = obs
        
        # Create initial variables
        self.score = 0

        return obs
    
    def preprocess(self, observation): 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (84,84,1))
        return state
    
    def close(self): 
        self.game.close()

In [16]:
env = StreetFighter()

In [17]:
env.observation_space.shape

(84, 84, 1)

# Hyperparameter tune

In [18]:
# !pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [19]:
# !pip install stable-baselines3[extra] optuna

In [20]:
import os
# Import optuna for HPO
import optuna
# Import PPO for algos
from stable_baselines3 import PPO
# Evaluate Policy
from stable_baselines3.common.evaluation import evaluate_policy
# Import wrappers
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

/Users/jacobjun/Python projects/Street-Fighter-agent/street_fighter/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
LOG_DIR = './logs/'
OPT_DIR = './opt_nodelta/'

In [22]:
# #https://github.com/araffin/rl-baselines-zoo/issues/29
def optimize_ppo(trial):
    """ Learning hyperparamters we want to optimise"""
    return {
        'n_steps': trial.suggest_int('n_steps', 2048, 8192),
        'gamma': trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range': trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda': trial.suggest_uniform('gae_lambda', 0.8, .99)
    }

In [23]:
def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial)
        env = StreetFighter()
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        model.learn(total_timesteps=100000)
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=20)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)
        return mean_reward
    except Exception as e: 
        return -1000

In [24]:
# study = optuna.create_study(direction='maximize')
# study.optimize(optimize_agent, n_trials=100, n_jobs=1)

# Setup Callback

In [25]:
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [26]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [27]:
CHECKPOINT_DIR = './train_nodelta/'

In [28]:
callback = TrainAndLoggingCallback(check_freq=250000, save_path=CHECKPOINT_DIR)

# Train Model

In [29]:
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [30]:
model_params = {'n_steps': 2570.949, 'gamma': 0.906, 'learning_rate': 2e-07, 'clip_range': 0.369, 'gae_lambda': 0.891}
#model_params = {'n_steps': 8960, 'gamma': 0.906, 'learning_rate': 2e-03, 'clip_range': 0.369, 'gae_lambda': 0.891}
# model_params = study.best_params

In [31]:
model_params['n_steps'] = 40*64

In [32]:
model_params

{'n_steps': 2560,
 'gamma': 0.906,
 'learning_rate': 2e-07,
 'clip_range': 0.369,
 'gae_lambda': 0.891}

In [33]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)
#model.load('./train_nodelta_backup/best_model_5460000.zip')

Using cpu device
Wrapping the env in a VecTransposeImage.


In [34]:
# model.learn(total_timesteps=10000000, callback=callback)
model.learn(total_timesteps=1000)

Logging to ./logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 154  |
|    iterations      | 1    |
|    time_elapsed    | 16   |
|    total_timesteps | 2560 |
-----------------------------


In [35]:
env.close()

# Evaluate the Model

In [36]:
# model = PPO.load('./train/best_model_90000')

In [37]:
# env = StreetFighter()
# env = Monitor(env, LOG_DIR)
# env = DummyVecEnv([lambda: env])
# env = VecFrameStack(env, 4, channels_order='last')

In [38]:
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=1, render=True)

In [ ]:
# mean_reward

In [ ]:
# env.close()

# Test out the Model

In [ ]:
import time

In [ ]:
# for episode in range(1): 
#     obs = env.reset()
#     done = False
#     total_reward = 0
#     while not done: 
#         action, _ = model.predict(obs)
#         obs, reward, done, info = env.step(action)
#         env.render()
#         time.sleep(0.01)
#         total_reward += reward
#     print('Total Reward for episode {} is {}'.format(total_reward, episode))
#     time.sleep(2)